In [63]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Path to trained weights file
#COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

### Configuration

In [64]:
class EarConfig(Config):
    """Configuration for training on the AWE dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "ear"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + ear

    # Number of training steps per epoch
    #STEPS_PER_EPOCH = 50

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    #BACKBONE = "resnet50"


### Dataset

In [65]:
class EarDataset(utils.Dataset):

    def load_ear(self, dataset_dir, subset):
        """Load a subset of the AWE ear dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or test    
        """
        
        # Add classes. We only have one class.
        self.add_class("ear", 1, "ear")
        
        # Assert folders train and test in dataset path
        assert subset in ["train", "test"]
        dataset_dir = os.path.join(dataset_dir, subset)
        mask_dir = os.path.join(dataset_dir, "trainannot")
        
        images = []
        for (dirpath, dirnames, filenames) in walk(dataset_dir):    
            images.extend(filenames)       
            break
                
        for filename in images:
            image_path = os.path.join(dataset_dir, filename)
            mask_path = os.path.join(mask_dir, filename)
            img = skimage.io.imread(image_path)
            height, width = img.shape[:2]            
            self.add_image(
                "ear",
                image_id=os.path.splitext(filename)[0],
                path=image_path,
                width=width, 
                height=height, 
                mask=mask_path
            )
        
    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a ear dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "ear":
            return super(self.__class__, self).load_mask(image_id)
        
        # Get a mask per instance        
        info = self.image_info[image_id]
        #mask = np.zeros([info["height"], info["width"], (#number_instances)], dtype=np.uint8)
        #mask = np.zeros([info["height"], info["width"], 1], dtype=np.uint8)
        
        # Read mask files from .png image
        mask = []
        #for i in instances  
        m = skimage.io.imread(info["mask"]).astype(np.bool)
        mask.append(m)
        
        mask = np.stack(mask, axis=-1)
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "ear":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

### Training

In [66]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = EarDataset()
    dataset_train.load_ear(args.dataset, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = EarDataset()
    dataset_val.load_ear(args.dataset, "test")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')